In [2]:
%pip install autodistill \
autodistill-grounded-sam \
supervision==0.9.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.4/82.4 KB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 16.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 28.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 KB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.3 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 KB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 8.1 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.4/770.4 KB 21.1 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 KB 14.1 MB/s eta 0:00:00
  Created wheel for rf-groundingdino: filename=rf_groundingd

In [8]:
%pip install tqdm ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
HOME = os.getcwd()
print(HOME)
!nvidia-smi

/app
Tue Jun 13 16:39:32 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.60.02    Driver Version: 526.98       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:07:00.0  On |                  N/A |
|  0%   45C    P5    19W / 200W |   1269MiB /  8192MiB |     32%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+--

In [4]:
import torch
torch.cuda.is_available()

True

In [5]:
from autodistill.detection import CaptionOntology
from autodistill_grounded_sam import GroundedSAM


ontology=CaptionOntology({
    "white curb": "white_curb",
    "red curb": "red_curb",
    "helmet":"helmet",
    "motorcycle wheel":"wheel",
    "motorbike":"moto",
    "motocycle":"moto",
    "road":"road"
})
base_model = GroundedSAM(ontology=ontology)

trying to load grounding dino directly
downloading dino model weights


/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


final text_encoder_type: bert-base-uncased


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
import os
print(os.listdir())

['.gitignore', '.vscode', '2019_americasGP', 'annotator.ipynb', 'compose-dev.yaml', 'dataset', 'docker-compose.yml', 'Dockerfile', 'helpers.py', 'live_inference.py', 'models', 'requirements.txt', 'video', 'yolo_inference.ipynb']


In [8]:
import cv2
import supervision as sv
from tqdm import tqdm_notebook as tqdm

SOURCE_IMAGE_PATH="./2019_americasGP/"
ontology=CaptionOntology({
    "white curb": "curb",
    "red curb": "curb",
    "helmet":"helmet",
    "wheel":"wheel",
    "motorbike":"moto",
    "motocycle":"moto",
    "motorcycle rider":"rider",
    "road":"road"
})

classes=ontology.classes()
print(classes)
base_model.ontology=ontology
base_model.box_threshold = 0.4
base_model.text_threshold = 0.5

imgs = sv.list_files_with_extensions(directory=SOURCE_IMAGE_PATH)

print(len(imgs))

for im in tqdm(imgs[300:301]):
  im_path=str(im.absolute())
  print(im_path)
  image = cv2.imread(im_path)

  detections = base_model.predict(input=im_path)
  #print(detections)
  
 
  box_annotator = sv.BoxAnnotator()
  mask_annotator = sv.MaskAnnotator()
  annotated_frame = box_annotator.annotate(scene=image.copy(), detections=detections,labels=[classes[cls_id] for cls_id in detections.class_id])
  image = mask_annotator.annotate(scene=annotated_frame, detections=detections,opacity=0.8)
  h, w, c = image.shape
  #image = image[0:h, 220:w]
  sv.plot_image(image, (16, 16))

['curb', 'curb', 'helmet', 'wheel', 'moto', 'moto', 'rider', 'road']
1397


/tmp/ipykernel_3119/2208928599.py:27: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for im in tqdm(imgs[300:301]):


  0%|          | 0/1 [00:00<?, ?it/s]

/app/2019_americasGP/2019_AmericasGP_1268.jpg


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:884: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:459: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


OutOfMemoryError: CUDA out of memory. Tried to allocate 1024.00 MiB (GPU 0; 8.00 GiB total capacity; 5.24 GiB already allocated; 0 bytes free; 6.33 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
MODEL_PATH="/home/jovyan/work/models/yolov8n-seg-150e.pt"
INPUT=SOURCE_IMAGE_PATH+

In [ ]:
%yolo predict model={TRAINED_MODEL_PATH} source={INPUT_VIDEO_PATH}